# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [60]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [61]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [62]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [63]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [64]:
# Getting the data from the top 10 expensive neighbourhoods
to_data_mean = to_data.reset_index().copy().drop(columns=["year"]).set_index("neighbourhood").groupby("neighbourhood").mean().sort_values("average_house_value", ascending=False)

# Calculate the mean number of dwelling types units per year
to_data_groupby = to_data.groupby("year").sum().drop(columns=["average_house_value", "shelter_costs_owned", "shelter_costs_rented"])

# Calculate the average monthly shelter costs for owned and rented dwellings
to_data_shelter_costs = (to_data[["shelter_costs_owned", "shelter_costs_rented"]]).groupby('year').mean()

### Panel Visualization Functions

In [65]:
# Define Panel visualization functions

def neighbourhood_map():
    px.set_mapbox_access_token(map_box_api)
    to_data_mean_neighbour = to_data_mean.sort_values("neighbourhood")
    df_locations_mean = pd.merge(df_neighbourhood_locations, to_data_mean_neighbour, on="neighbourhood", how="inner")
    map_info = px.scatter_mapbox(
        df_locations_mean,
        lat="lat",
        lon="lon",
        hover_data=df_locations_mean,
        hover_name="neighbourhood",
        size="average_house_value",
        color="average_house_value",
        color_continuous_scale="icefire",
        title="Average House Value in Toronto",
        zoom=9,
        width=900, 
        height=500)
    plotly_panel = pn.pane.Plotly(map_info)
    plotly_panel._updates = True
    return plotly_panel

def create_bar_chart(data, title, xlabel, ylabel, color):
    to_data_type = data.hvplot.bar(
    title=(title),
    xlabel=(xlabel), 
    ylabel=(ylabel),
    color=(color),
    rot=90,
    width=600, height=500).opts(
    yformatter="%.0f")
    #plotly_panel = pn.pane.Plotly(to_data_type)
    #plotly_panel._updates = True
    return to_data_type

def create_line_chart(data, title, xlabel, ylabel, color):
    to_data_shelter = data.hvplot.line(
    title=(title),
    xlabel=(xlabel), 
    ylabel=(ylabel),
    color=(color),
    width=550, 
    height=350)
    plotly_panel = pn.pane.Plotly(to_data_shelter)
    plotly_panel._updates = True
    return to_data_shelter

average_house_value = (to_data[["average_house_value"]]).groupby("year").mean().sort_values("year")
def average_house_value():
    average_house_value_fig = average_house_value.hvplot.line(
        x="year",
        y="average_house_value",
        xlabel="Year", 
        ylabel="Avg House Value",
        title="Average House Value in Toronto",
        width=480, 
        height=350,
    ).opts(line_color='blue', yformatter="%.0f")
    plotly_panel = pn.pane.Plotly(average_house_value_fig)
    plotly_panel._updates = True
    return average_house_value_fig

avg_per_neighb = (to_data[["neighbourhood", "average_house_value"]]).sort_values(["year","neighbourhood"]).reset_index()
list_of_neighbourhood = avg_per_neighb["neighbourhood"].values.tolist()

def choose_neighbourhood(neighbourhood):
    return neighbourhood

def interact_neighbourhood():
    interact(choose_neighbourhood, neighbourhood=list_of_neighbourhood)
    return interact

def average_value_by_neighbourhood():
    avg_per_neighb_plot = avg_per_neighb.hvplot.line(
        x="year",
        y="average_house_value",
        xlabel="Year", 
        ylabel="Avg House Value",
        title="Neighbourhood:",
        groupby='neighbourhood',
        width=600, 
        height=450).opts(line_color='blue', yformatter="%.0f")
    plotly_panel = pn.pane.Plotly(avg_per_neighb_plot)
    plotly_panel._updates = True
    return avg_per_neighb_plot

to_data_all = to_data.reset_index()
to_data_all = to_data_all[["year", "neighbourhood", "single_detached_house", "apartment_five_storeys_plus", "movable_dwelling", "semi_detached_house", "row_house", "duplex", "apartment_five_storeys_less", "other_house"]].set_index("year")
def number_dwelling_types():
    dwelling_types = to_data_all.hvplot.bar(
        xlabel="Year", 
        ylabel="Dwelling Types Units",
        title="Neighbourhood:",
        groupby='neighbourhood',
        rot=90,
        width=600, 
        height=450).opts(line_color='blue', yformatter="%.0f")
    #plotly_panel = pn.pane.Plotly(dwelling_types)
    #plotly_panel._updates = True
    return dwelling_types

def average_house_value_snapshot():
    fig = px.bar(
        to_data_year, 
        x="neighbourhood", 
        y="average_house_value", 
        color="average_house_value", 
        facet_row="year",
        facet_row_spacing=0.04,
        facet_col_spacing=0.04,
        height=1000, width=900,
        title="Average House Values in Toronto per Neighbourhood")
    #plotly_panel = pn.pane.Plotly(fig)
    #plotly_panel._updates = True
    return fig

def top_most_expensive_neighbourhoods():
    to_data_exp_top10 = to_data_mean.iloc[0:10, :].hvplot.bar(
    x='neighbourhood', 
    y='average_house_value',
    xlabel="Neighbourhood", 
    ylabel="Avg House Value",
    title="Top 10 Expensive Neighbourhood in Toronto",
    rot=90,  
    width=600, 
    height=500).opts(
    yformatter="%.0f")
    #plotly_panel = pn.pane.Plotly(to_data_exp_top10)
    #plotly_panel._updates = True
    return to_data_exp_top10

def create_line_chart_shelter(data, title, xlabel, ylabel, color):
    to_data_shelter_costs = data.hvplot.line(
    title=(title),
    xlabel=(xlabel), 
    ylabel=(ylabel),
    color=(color),
    width=500, height=300)
    return to_data_shelter_costs

to_data_expensive = to_data.loc[to_data["average_house_value"] >= 600000, :].reset_index()
def sunburts_cost_analysis():
    fig_1 = px.sunburst(to_data_expensive, path=["year", "neighbourhood"], values="average_house_value",
                  color="shelter_costs_owned", hover_data=["average_house_value"],
                  color_continuous_scale="Blues", title="Costs Analysis of Most Expensive Neighbourhood in Toronto per Year",
                  color_continuous_midpoint=np.average(to_data_expensive["shelter_costs_owned"], 
                                                       weights=to_data_expensive["shelter_costs_rented"]))
    plotly_panel = pn.pane.Plotly(fig_1)
    plotly_panel._updates = True
    return plotly_panel

In [66]:
# Bar chart for 2001
create_bar_chart_2001 = create_bar_chart(
    data = to_data_groupby.iloc[0],
    title="Dwelling Types Units in Toronto in 2001",
    xlabel="2001", 
    ylabel="Dwelling Type Units",
    color="red",
    )

# Bar chart for 2006
create_bar_chart_2006 = create_bar_chart(
    data = to_data_groupby.iloc[1],
    title="Dwelling Types Units in Toronto in 2006",
    xlabel="2006", 
    ylabel="Dwelling Type Units",
    color="blue",
    )

# Bar chart for 2011
create_bar_chart_2011 = create_bar_chart(
    data = to_data_groupby.iloc[2],
    title="Dwelling Types Units in Toronto in 2011",
    xlabel="2011", 
    ylabel="Dwelling Type Units",
    color="orange",
    )

# Bar chart for 2016
create_bar_chart_2016 = create_bar_chart(
    data = to_data_groupby.iloc[3],
    title="Dwelling Types Units in Toronto in 2016",
    xlabel="2016", 
    ylabel="Dwelling Type Units",
    color="magenta",
    )


In [67]:

create_line_shelter_owned = create_line_chart_shelter(
    data = to_data_shelter_costs.iloc[:,0],
    title="Average Monthly Shelter Cost for Owned Dwellings in Toronto",
    xlabel="Year", 
    ylabel="Avg Monthly Shelter Costs",
    color="blue",
    )

create_line_shelter_rented = create_line_chart_shelter(
    data = to_data_shelter_costs.iloc[:,1],
    title="Average Monthly Shelter Cost for Rented Dwellings in Toronto",
    xlabel="Year", 
    ylabel="Avg Monthly Shelter Costs",
    color="orange",
    )

# Calculate the average house value per year
average_house_value = (to_data[["average_house_value"]]).groupby("year").mean()

average_house_value.hvplot.line(
    x="year",
    y="average_house_value",
    xlabel="Year", 
    ylabel="Avg House Value",
    title="Average House Value in Toronto",
    width=480, 
    height=350,
    ).opts(line_color="blue", yformatter="%.0f")

:Curve   [year]   (average_house_value)

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [68]:
# Create a Title for the Dashboard
dashboard_title = "Toronto Real Estate Analysis from 2001 to 2016"

# Define a welcome text
welcome_column = pn.Column(
    "This Dashbord presents a visual analysis of historical house values, dwelling types per neighbourhood and dwelling costs in Toronto, Ontario according to census data from 2001 to 2016. You can navigate through the tabs above to explore more details about the evolution of the real estate market for these years.",
        neighbourhood_map)

# Create the main dashboard
row_of_dwelling_types = pn.Row(create_bar_chart_2001 + create_bar_chart_2006) 
row_of_dwelling_types_2 = pn.Row(create_bar_chart_2011 + create_bar_chart_2016)
mkt_column = pn.Column(row_of_dwelling_types, row_of_dwelling_types_2)
#costs_column = pn.Column(create_line_shelter_owned, create_line_shelter_rented, average_house_value_snapshot)

#column_of_analysis_1 = pn.Column(average_value_by_neighbourhood, number_dwelling_types)
#column_of_analysis_2 = pn.Column(average_house_value_snapshot)

#row_of_analysis = pn.Row(column_of_analysis_1, column_of_analysis_2)
#row_of_top_exp = pn.Row(top_most_expensive_neighbourhoods, sunburts_cost_analysis)

# Create a tab layout for the dashboard
dashboard = pn.Column(dashboard_title,
                      pn.Tabs(
                          ("Welcome", welcome_column),
                         #("Yearly Market Analysis", mkt_column),
                         #("Shelter Costs Vs. House Value", costs_column),
                          #("Neighbourhood Analysis", row_of_analysis),
                          #("Top Expensive Neighbourhood", row_of_top_exp)
))                          

## Serve the Panel Dashboard

In [69]:
dashboard.servable()

Column
    [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Markdown(str)
            [1] Column
                [0] Column()
                [1] Row
                    [0] Plotly(Figure)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [70]:
#neighbourhood_map().show()

In [71]:
#create_bar_chart(to_data, "title", "abc", "def", "blue")

# Bar chart for 2001
#create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [72]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
# average_house_value()

In [ ]:
# average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [ ]:
# average_house_value_snapshot()

In [ ]:
# top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()